In [ ]:
#Install Dependencies
!pip install telebot
!pip install langchain_community
!pip install chromadb
!pip install bitsandbytes
!pip install googletrans==3.1.0a0

In [ ]:
#Import Necessary Libraries
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from PIL import Image
import requests
from io import BytesIO
import IPython

import pandas as pd


import telebot
from googletrans import Translator

In [ ]:
#Login to huggingface to get the Gated Model
from huggingface_hub import login
login()
##  YOUR-API-KEY

In [ ]:
#Translation Examples
translator = Translator()

txt = "من یک زن ورزشکار و با قد 171 سانتی متر در اواسط دهه بیستم هستم، بدنی مستطیل شکل با شانه های کمی پهن و استایلی براق و غیررسمی دارم. من معمولا رنگ های تیره تر را ترجیح می دهم."
translation = translator.translate(txt) #,src="auto",dest='en')
print("fa->en: ",translation)

translation = translator.translate("I am a 171cm tall athletic woman.")
print("en->?: ",translation)

In [ ]:
#Download Online-Shop Products Dataset. (CSV File)
url = "https://raw.githubusercontent.com/aix64-main/LLM_Fashion_Telegram/refs/heads/main/Myntra_fashion_products.csv"
products_df = pd.read_csv(url)
print(products_df.head(3))

In [ ]:
#Convert CSV Data to doc<->meta-data Pairs
docs = []
m_data = []

for index, row in products_df.iterrows():
    td = f"For {row['gender']} - {row['name']} - {row['description']}"
    docs.append(td)
    m_data.append({"index_in_db":index,
                   "images":row['images'],
                   "price":row['price'],
                   "name":row['name'],})
idx = 11
print(docs[idx])
print(m_data[idx])

In [ ]:
# Model for creating the embeddings
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device":device}
)

In [ ]:
#Put doc<->meta-data info into VectorDB
persist_directory = "./chromadb"
vectordb = Chroma.from_texts(texts=docs,
                             metadatas=m_data,
                             embedding=embedding,
                             persist_directory=persist_directory)

In [ ]:
#Print Data Stored in VectorDB
vdb = vectordb.get(include=['embeddings', 'documents', 'metadatas'])
num=2
print(vdb['documents'][:num])
print(vdb['embeddings'][:num])
print(vdb['metadatas'][:num])
print(len(vdb['documents']))

In [ ]:
#Example of using the Retrieval Model
prompt = "For Women, Shoe: Sleek loafers in a contrasting color like burgundy or deep green."
documents  = vectordb.similarity_search(prompt,k=3)
print(documents)

In [ ]:
#Functions for Image Processing Tasks and Products Retrieval and Recommendation
def get_image_by_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

def concat_images_h(images):
    width, height = images[0].size
    total_width = width*len(images)
    new_im = Image.new('RGB', (total_width, height))
    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]
    return new_im

def concat_images_v(images):
    width, height = images[0].size
    total_height = height*len(images)
    new_im = Image.new('RGB', (width, total_height))
    y_offset = 0
    for im in images:
      new_im.paste(im, (0,y_offset))
      y_offset += im.size[1]
    return new_im


def retrieval_products(prompt):
    documents  = vectordb.similarity_search(prompt,
                                            k=3)

    tmp_imgs_v = []
    tmp_imgs_info = []
    for cnt,doc in enumerate(documents):
        idx_in_db = doc.metadata["index_in_db"]
        price = doc.metadata["price"]
        images = doc.metadata["images"]
        name = doc.metadata["name"]
        images = images.split("~")
        images = [img.strip() for img in images]
        tmp_imgs_h = []
        num_valid_img = 0
        for url in images:
            try:
                img = get_image_by_url(url)
                img = img.resize((256,256))
                tmp_imgs_h.append(img)
                num_valid_img += 1
            except:
                pass
            if num_valid_img>=3:
                break
        img = concat_images_h(tmp_imgs_h)
        tmp_imgs_v.append(img)
        info = "ردیف "+str(cnt+1)+":\n"
        info += "قیمت: "+str(price)+"\n"
        info += "نام محصول: "+str(name)+"\n"
        info += "شناسه: "+str(idx_in_db)+"\n"
        info += "----------------------------\n"
        tmp_imgs_info.append(info)

    final_img = concat_images_v(tmp_imgs_v)
    final_txt = "محصولات پیشنهادی ما برای شما به این صورت است: \n\n"
    final_txt += "\n".join(tmp_imgs_info)
    return final_img,final_txt

In [3]:
#An Example for Products Recommendation
img,txt = retrieval_products(prompt)
img = img.resize((256,256))
display(img)
print(txt)

In [5]:
#Get the LLM Model for Outfit Recommendation Task
#Prepare Model Prompts
def format_instruction(input, event):
    return f"""You are a personal stylist recommending fashion advice and clothing combinations. Use the self body and style description below, combined with the event described in the context to generate 5 self-contained and complete outfit combinations.
        ### Input:
        {input}

        ### Context:
        I'm going to a {event}.

        ### Response:
    """

# input is a self description of your body type and personal style
prompt_r = "I'm an athletic and 171cm tall woman in my mid twenties, I have a rectangle shaped body with slightly broad shoulders and have a sleek, casual style. I usually prefer darker colors."
event = "business meeting"
prompt = format_instruction(prompt_r, event)
print(prompt)

In [6]:
#Get the Model and it's Tokenizer
# load base LLM model, LoRA params and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    "neuralwork/mistral-7b-style-instruct",
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained("neuralwork/mistral-7b-style-instruct")


def outfit_recommendation(prompt_r,event):
    prompt = format_instruction(prompt_r, event)
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    # inference
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=600,
            do_sample=True,
            top_p=0.9,
            temperature=0.9
        )

    # decode output tokens and strip response
    outputs = outputs.detach().cpu().numpy()
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    output = outputs[0][len(prompt):]
    return output

In [ ]:
#Test the Outfit Recommender Model
output = outfit_recommendation(prompt_r,event)
print(output)

In [7]:
#Translate Outfit Recommender Results
def translate_recom(recom):
    translation = translator.translate(recom,src="en",dest='fa')
    trecom = translation.text
    trecom = trecom.replace("لوازم جانبی:", "اکسسوری:")
    return trecom

final_recom = translate_recom(output)
print(final_recom)

In [8]:
#Telegram Bot
#A Function for Splitting Outfits

import re

def seperate_outfits(txt):
    all_outfits = []
    res = re.split(r'[0-9]\.\s', txt)
    res = [r.strip() for r in res if r.strip()!=""]
    for _rs in res:
        outfit = [_r for _r in _rs.splitlines() if (_r!="" and not _r.startswith("Outfit"))]
        all_outfits.append(outfit)
    return all_outfits

In [9]:
#Telegram Bot Respone Buttons
from telebot import types

bt_1 = types.InlineKeyboardButton('گزینه 1', callback_data='outfit_1')
bt_2 = types.InlineKeyboardButton('گزینه 2', callback_data='outfit_2')
bt_3 = types.InlineKeyboardButton('گزینه 3', callback_data='outfit_3')
bt_4 = types.InlineKeyboardButton('گزینه 4', callback_data='outfit_4')

outfit_keyboard = types.InlineKeyboardMarkup()
outfit_keyboard.add(bt_1,bt_2)
outfit_keyboard.add(bt_3,bt_4)

In [ ]:
#Main Telegram Bot Message (for /help command)
main_msg = """
سلام
من دستیار Admin هستم. به دو روش میتونم به شما کمک کنم.

1. برای پیشنهاد محصولات جهت خرید، به این صورت به من پیام بدین:
/buy
جنسیت (خانم/آقا)
توضیحات در مورد وسیله مورد نیاز

به عنوان مثال:
---------------------------------
/buy
خانم
کفش براق در رنگ های متضاد شرابی یا سبز تیره
---------------------------------




2. برای پیشنهاد مجموعه outfit به این صورت به من پیام بدین:
/recom
جنسیت (خانم/آقا)
رویداد (جلسه کاری، مهمانی تولد، مهمانی عروسی و ...)
توضیحات در رابطه با شرایط و استایل و رنگ مورد علاقه و فیزیک بدنی خودتان

به عنوان مثال:
---------------------------------
/recom
خانم
جلسه کاری
من یک زن ورزشکار و با قد 175 سانتی متر حدودا 25 ساله هستم. بدنی مستطیل شکل با شانه های کمی پهن و استایلی غیررسمی دارم. من معمولا رنگ های تیره تر را ترجیح می دهم.
---------------------------------

"""

In [ ]:
#Main Codes for the Telegram Bot
API_TOKEN = "YOUR-API-TOKEN"
bot = telebot.TeleBot(API_TOKEN)

main_outfit_recom = ""
main_gender_recom = ""


## Handle '/help'
@bot.message_handler(commands=['help'])
def send_welcome(message):
    msg = bot.reply_to(message,main_msg,
                       parse_mode='html')
    
@bot.message_handler(commands=['start'])
def send_welcome(message):
    msg = bot.reply_to(message,main_msg,
                       parse_mode='html')


## Handle '/recom'
@bot.message_handler(commands=['recom'])
def send_welcome(message):
    global main_outfit_recom
    global main_gender_recom

    data = message.text.splitlines()
    if ("آقا" in data[1]) or ("اقا" in data[1]) or ("مرد" in data[1]):
        gender_txt = "I am a Man. "
    else:
        gender_txt = "I am a Woman. "
    main_gender_recom = data[1]
    event = data[2]
    translation = translator.translate(event)
    event = translation.text

    prompt_txt = " ".join(data[3:])
    translation = translator.translate(prompt_txt)
    prompt_txt = translation.text

    prompt_r = gender_txt + prompt_txt
    or_recom = outfit_recommendation(prompt_r,event)
    or_recom = or_recom.split("5. Outf")[0]
    main_outfit_recom = or_recom

    tr_recom = translate_recom(or_recom)
    txt = "Original Respone:\n"
    txt+= "-------------------\n"
    txt+= or_recom+"\n\n\n"
    txt+= "پاسخ ترجمه شده (توسط ماشین):\n"
    txt+= "-------------------\n"
    txt+= tr_recom+"\n\n"
    msg = bot.reply_to(message,txt, parse_mode='html',reply_markup=outfit_keyboard)



## Handle '/buy'
@bot.message_handler(commands=['buy'])
def send_welcome(message):
    data = message.text.splitlines()
    if ("آقا" in data[1]) or ("اقا" in data[1]) or ("مرد" in data[1]):
        gender_txt = "For Men, "
    else:
        gender_txt = "For Women, "
    prompt_txt = " ".join(data[2:])
    translation = translator.translate(prompt_txt)
    prompt_txt = translation.text
    prompt = gender_txt + prompt_txt
    img, txt = retrieval_products(prompt)
    chat_id = message.chat.id
    msg = bot.send_photo(chat_id,img,
                         caption=txt,
                         reply_to_message_id=message.id)



def send_outfit_recom(message,prompt,caption):
    img, txt = retrieval_products(prompt)
    chat_id = message.chat.id
    caption+= "\n************************\n\n\n"
    caption+= txt
    msg = bot.send_photo(chat_id,img,
                         caption=caption,
                         reply_to_message_id=message.id)




@bot.callback_query_handler(func=lambda call: True)
def handle_query(call):
    global main_outfit_recom
    global main_gender_recom

    all_outfits = seperate_outfits(main_outfit_recom)
    if ("آقا" in main_gender_recom) or ("اقا" in main_gender_recom) or ("مرد" in main_gender_recom):
        gender_txt = "For Men, "
    else:
        gender_txt = "For Women, "

    index = int(call.data.split("_")[-1])-1
    outfit = all_outfits[index]
    for ofit in outfit:
        prompt = gender_txt + ofit
        send_outfit_recom(call.message,prompt,ofit)
    return

In [ ]:
#Run Telegram Bot (infinite loop)
print('TG bot Started . . .')
bot.infinity_polling()